In [74]:
import pandas as pd
import numpy as np

df = pd.read_csv("/Users/Yur/Downloads/ovitrap_data_with_province.csv", index_col=[0], usecols = ['date', 'name', 'value', 'adm'])

In [75]:
summary_df = df.groupby(['name']).describe()

In [76]:
summary_df.head(5)
print(len(summary_df)) # total number of schools

value                              \
                                      count       mean        std   min   
name                                                                      
A. Bonifacio Elementary School          7.0  39.251429  30.042547  0.00   
A. Deato Elementary School             28.0  22.266786  11.050519  0.00   
A. Fernando Elementary School          64.0  25.810313  21.189406  0.00   
A. J. Villegas Vocational High School  17.0  17.471176  20.248154  1.85   
A. Mabini Elementary School            47.0  43.625319  18.355262  0.00   

                                                                      
                                          25%    50%      75%    max  
name                                                                  
A. Bonifacio Elementary School         14.285  51.42  60.9600  72.85  
A. Deato Elementary School             20.830  25.00  27.0825  37.50  
A. Fernando Elementary School          11.540  19.72  37.0000  75.00  
A. J. Villegas Vocational High School  11.670  13.33  14.0000  92.59  
A. Mabini Elementary School            31.665  47.46  57.4300  77.27

In [77]:
summary_df.columns = summary_df.columns.droplevel(0)

In [78]:
summary_df = summary_df[summary_df['count'] > 10] # only look at schools with more than 10 observations
summary_df = summary_df[summary_df['max'] > 10] #filter schools with a max less than 10
summary_df = summary_df[summary_df['mean'] < 90] #filter school with absurdly high means (questionable step)
summary_df = summary_df_mean[~((summary_df['max']==100)&((summary_df['50%']==0)|(summary_df['50%']==100)))] #most 100 and 0 schools filtered

In [82]:
summary_df.index.tolist()

['A. Deato Elementary School',
 'A. Fernando Elementary School',
 'A. J. Villegas Vocational High School',
 'A. Mabini Elementary School',
 'A. Mirasol Elementary School',
 'A. Pichon Elementary School',
 'A. V. Mijares ES',
 'A.L. Navarro National High School',
 'A.P. Decano Memorial ES',
 'ATIMONAN NCHS',
 'Abaca ES',
 'Abanon Central Elementary School',
 'Abebeg PS',
 'Abkasa Elementary School',
 'Ablang ES',
 'Abo-Abo NHS*',
 'Abra High School',
 'Abucay National High School',
 'Actin High School',
 'Adamson ES',
 'Adgao-Tagpuan-Ingay Elementary School',
 'Adoyungan Primary School',
 'Aganan Ungka Elementary School',
 'Agape Elementary School',
 'Agcawilan Silakat Nonok ES',
 'Agmanaphao ES',
 'Agno National High School (HS)',
 'Agoncillo Elementary School',
 'Agoo West Central Elementary School',
 'Agsanayan National High School',
 'Agsungot Elementary School',
 'Aguinaldo Elementary School',
 'Agusuhin Elementary School ',
 'Agutayan-Lupa ES',
 'Ajuy National High School',
 'Akla

In [89]:
df = pd.read_csv("/Users/Yur/Downloads/ovitrap_data_with_province.csv", index_col=[0])

semifiltered_df = df[df['name'].isin(summary_df.index.tolist())]
export_csv = df.to_csv('semiFiltered.csv')

In [90]:
semifiltered_df

,date,id,latitude,longitude,name,value,adm
0,2012-06-19,3,14.504400,121.056000,Kapitan Jose Cardones Memorial School,17.310,"NCR, Fourth District"
1,2012-07-03,3,14.504400,121.056000,Kapitan Jose Cardones Memorial School,15.520,"NCR, Fourth District"
2,2012-07-17,3,14.504400,121.056000,Kapitan Jose Cardones Memorial School,30.000,"NCR, Fourth District"
3,2012-07-24,3,14.504400,121.056000,Kapitan Jose Cardones Memorial School,22.640,"NCR, Fourth District"
4,2012-09-11,3,14.504400,121.056000,Kapitan Jose Cardones Memorial School,28.130,"NCR, Fourth District"
5,2012-09-18,3,14.504400,121.056000,Kapitan Jose Cardones Memorial School,25.760,"NCR, Fourth District"
6,2012-09-25,3,14.504400,121.056000,Kapitan Jose Cardones Memorial School,15.710,"NCR, Fourth District"
7,2012-10-02,3,14.504400,121.056000,Kapitan Jose Cardones Memorial School,18.750,"NCR, Fourth District"
8,2012-10-09,3,14.504400,121.056000,Kapitan Jose Cardones Memorial School,21.540,"NCR, Fourth District"
9,2012-10-16,3,14.504400,121.056000,Kapitan Jose Cardones Memorial School,26.320,"NCR, Fourth District"
